In [1]:
from twikit import Client, TooManyRequests
from account_info import USERNAME, EMAIL, PASSWORD
import pandas as pd
import json
import urllib.request
import os
import time
import requests
import re



# Initialize client
client = Client('en-US')

client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD
)

# save cookies in order to pull data without getting banned
client.save_cookies('cookies.json')

In [2]:
# loads the cookies that were saved
client.load_cookies('cookies.json')

In [3]:
save_path_list = []

# all the file paths for all the structures in the end 
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\user_structure.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\post_structure.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\comment_structure.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\picture_strucutre.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\keyword_structure.json')
save_path_list.append(r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\data_structures\relation_structure.json')

# file path to excel sheet
file_path_to_excel = r"C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\codes.xlsx"

In [15]:
def download_image(URL, save_as):
    '''Function uses image URL and saves the image onto the desired path and file type'''
    urllib.request.urlretrieve(URL,save_as)


def delete_jpg(file):
    '''This file takes in the file path and deletes it.'''
    if os.path.exists(file):
        os.remove(file)
    else:
        print('File not found')


# this saves the json file on the described directory 
def save_json(file, file_path):
    with open(file_path, "w") as f:
        json.dump(file, f,indent=4)


def open_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data 


def retry_on_rate_limit_error(func, *args, **kwargs):
    max_retries = 5
    retries = 0
    while retries < max_retries:
        try:
            return func(*args, **kwargs)
        except TooManyRequests as e:
            print("Rate limit exceed trying again in 60 sec")
            time.sleep(60)
            retries += 1
    raise Exception("Max tries reached")


def keywords_to_dict(file_path =r"C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\codes.xlsx" ):
    '''This function takes in the excel sheet that has keywords and its ids split up into columns.
    Then it creates two dictionaries, keyname and keyname_id. The values are numbered 0 to the 
    length of the columns. Once created they return these dictionaries.'''

    # reads in excel sheet into pandas data frame
    df = pd.read_excel(file_path)

    # this turns the street name and its code name into list 
    street_name_list = df['keyname'].astype(str).values.tolist()
    street_name_code_list = df['keyname_id'].astype(str).values.tolist()

    # this dictionary contains the keyname in the keys and keyname id in values
    keyname = {}
    keyname_id = {}
    for i in range(26):
        keyname[i] = street_name_list[i]
        keyname_id[i] = street_name_code_list[i]
        
    return keyname, keyname_id


def keyword_search_in_users(keyname_to_search, list_of_users, list_to_append, keywords_dict, iteration):
    """This function takes in a keyname to search a list of users. It will search the users name,
    keyname, description and tweets for the keyname. If it finds it on the tweets it appends
    to the list_to_append"""

    keynames, keyname_ids = keywords_to_dict()
    keyword_id = keyname_ids[iteration]
    # gets a user from the list_of_users and goes through their name, screen name 
    # and description to find keywords. If they find one they append the user to the list]
    
    for user in list_of_users:
        if len(list_to_append) == 30:
            break
        else: 
            if keyname_to_search in user.name:
                keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                list_to_append.append('user_' +  user.id)

            elif keyname_to_search in user.screen_name:
                keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                list_to_append.append('user_' + user.id)

            elif keyname_to_search in user.description:
                keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                list_to_append.append('user_' + user.id)

    return list_to_append, keywords_dict


def keyword_search_in_tweets(tweet):
    """This function takes in tweets, keyname, and keyname ids. Then 
    it goes thourgh the keynames and trys to find it in the text of the 
    tweet. If it finds it appends it to a list and at the end it returns this
    list."""

    keyname, keyname_id = keywords_to_dict()
    keyname_found_in_tweet = []

    # find keyword in tweet text, if so append to list
    for keys, keyword in keyname.items():
        if keyword in tweet.full_text:
            keyname_found_in_tweet.append(keyname_id[keys])
    
    return keyname_found_in_tweet


def relations_in_comments(tweet, reply, relations):
            
            Relation_Structure = {
                "src_id": 'tweet_' + str(reply.id),
                "relation": 'comment-under-post',
                "dest_id": 'tweet_' + str(tweet.id)
            }

            reply_user = reply.user
            relations.append(Relation_Structure)

            Relation_Structure = {
                "src_id": 'user_' + str(reply_user.id),
                "relation": 'user-make-comment',
                "dest_id": 'tweet_' + str(reply.id)
            }

            relations.append(Relation_Structure)

            keywords, keywords_ids = keywords_to_dict()

            for key, keyname in keywords.items():
                if keyname in reply.full_text:
                    Relation_Structure = {
                        "src_id": 'tweet_' + str(reply.id),
                        "relation": 'comment-contain-keyword',
                        "dest_id": keywords_ids[key]
                    }

                    relations.append(Relation_Structure)
            return relations


def remove_duplicates_from_list(list):
    """This functions removes duplicates from list"""
    result = []
    for item in list:
        if item not in result:
            result.append(item)
    return result


def tweet_mentions_user(user, tweet, relations):
    """This function is for when a user mentions another user in a tweet. Finds 
    mentioned users user_name and adds the relation_structure to relations."""

    mentions = re.findall(r'@(\S+)', tweet.full_text)
    mentions = remove_duplicates_from_list(mentions)
    for mention in mentions:
        try:
            mentioned_user = client.get_user_by_screen_name(mention)
            Relation_Structure = {
                "src_id": 'user_' + str(user.id),
                "relation": 'user-mention/tag-user',
                "dest_id": 'user_' + str(mentioned_user.id)
            }

            relations.append(Relation_Structure)
        except:
            pass
    return relations 


def extract_tweet_picture_structure(user, tweet, pic_id_starter, relations):
    """This function takes in the type user and tweet. Then goes down the post
    structure sorting out the data. Finally it returns the post_structure"""
    
    if '@' in tweet.full_text:
        relations = tweet_mentions_user(user, tweet, relations)

    # retrieve keywords found in tweet
    keyname_found = keyword_search_in_tweets(tweet)
    # construct post structure
    favoriters = tweet.get_favoriters(20)

    Relation_Structure = {
            "src_id": 'user_' + str(user.id),
            "relation": 'user-publish-post',
            "dest_id": 'tweet_' + str(tweet.id)
        }

    relations.append(Relation_Structure)

    Post_Structure = {
        "user_id": 'user_' + str(user.id),
        "post_id": 'tweet_' + str(tweet.id),
        "user_comment": tweet.full_text,
        "pic_id": "",
        "liked_users": [ 'user_' + str(favoriter.id) for favoriter in favoriters],
        "comments": '',
        "keywords": keyname_found,
    }

    tweet = client.get_tweet_by_id(tweet.id)
    replies = tweet.replies
    if replies:
        Post_Structure["comments"] = [f'tweet_{reply.id}' for reply in replies]
        
        for reply in replies:
            relations = relations_in_comments(tweet, reply, relations)
            
    for favoriter in favoriters:
        Relation_Structure = {
                "src_id": 'user_' + str(favoriter.id),
                "relation": 'user-like-post',
                "dest_id": 'tweet_' + str(tweet.id)
            }
        
        relations.append(Relation_Structure)
         
    media_data = tweet.media
    if media_data:
        Post_Structure["pic_id"] = 'pic_' + str(pic_id_starter)

        Picture_Structure = {
            "pic_id": 'pic_' + str(pic_id_starter),
            "post_id": 'tweet_' + str(tweet.id),
            "url": media_data[0].get('media_url_https')
        }

        Relation_Structure = {
                "src_id": 'tweet_' + str(tweet.id),
                "relation": 'post-has-picture',
                "dest_id": 'post_' + str(pic_id_starter)
            }
        
        relations.append(Relation_Structure)
        pic_id_starter += 1
        return Post_Structure, Picture_Structure, pic_id_starter, relations
    
    else:
        Picture_Structure = {}
        return Post_Structure, Picture_Structure, pic_id_starter, relations


def get_relations_followers_following(relations, followers_list, user):
    '''This function gets the realtions between a user and it's
    following/follower list'''

    user_id = user.id
    for follower in followers_list:
        Relation_Structure = {
                "src_id": follower,
                "relation": 'user-follow/followed-user',
                "dest_id": 'user_' + str(user_id)
            }
                
        relations.append(Relation_Structure)
    return relations
    

def realtion_in_profile(user, keyword_id, relations):
    '''Finds the relations profile of users. If a keyword is found 
    in the profile it creates a relation structure for it.'''
    Relation_Structure = {
                "src_id": 'user_' + str(user.id),
                "relation": 'user-profile-keyword',
                "dest_id": keyword_id
            }
    relations.append(Relation_Structure)
    return relations


def realtion_keyword_in_tweet(tweet, keyword_id, relations):
    '''Adds relation structure between post and keyword'''
    Relation_Structure = {
                "src_id": 'tweet_' + str(tweet.id),
                "relation": 'post-include-keyword',
                "dest_id": keyword_id,
            }
    relations.append(Relation_Structure)
    return relations


def add_followers_or_following(followers_list):
    if len(followers_list) != 30:
        for follower in followers:
            if len(followers_list) == 30:
                break
            elif follower not in followers_list:
                followers_list.append('user_' + follower.id)
    return followers_list

In [ ]:

# all data structures 
users = {}
posts = {}
comments = {}
pictures = {}
keywords_dict = {}
relations = []

data_structures_to_save = [users,posts, comments, pictures, keywords_dict, relations]

for i in range(len(save_path_list)):
    save_json(data_structures_to_save[i], save_path_list[i])

In [13]:
keyname, keyname_id = keywords_to_dict()
search_key = keyname[0]
# [0,13] words to search. 
user_keyword = retry_on_rate_limit_error(client.search_user,'Amphetamine', 100)

# all data structures 
users = {}
posts = {}
comments = {}
pictures = {}
keywords_dict = {}
relations = []

key_structure_list = []
for keys, keynames in keyname.items():
    keyname_structure = {
        "keyword": keynames,
        "keyword_id": keyname_id[keys],
        "ids": [],
    }
    keywords_dict[keyname_id[keys]] = keyname_structure
    # key_structure_list.append(keyname_structure)

pic_id_num_starter = 20000000000
picture_id_dict = {}


account_num = 1
for user in user_keyword:
    # users = {}
    # posts = {}
    # comments = {}
    # pictures = {}
    # keywords_dict = {}
    # relations = []
    # if account_num % 20 == 0:
    #     print('sleeping')
    #     time.sleep(600)

    try: 

        keywords = []

        # keyname contains the keynames and keyname_id contains their ids
        keyname, keyname_id = keywords_to_dict()

        # get the users tweets
        tweets = user.get_tweets('tweets',15)

        # gets users followers returns a list of users, return user type 
        followers = user.get_followers(user.id)

        # gets users following returns a list of users, return user type
        following = user.get_following(user.id)

        
        # empty list for the followers
        followers_list = []
        following_list = []

        # add the user or their tweets to keyword_structure if a keyword is found
        for i in range(len(keyname)):
            keyword = keyname[i]
            keyword_id = keyname_id[i]
            if keyword in user.name:
                keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                relations = realtion_in_profile(user, keyword_id, relations)
                keywords.append(keyword_id)
            elif keyword in user.screen_name:
                keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                relations = realtion_in_profile(user, keyword_id, relations)
                keywords.append(keyword_id)
            elif keyword in user.description:
                keywords_dict[keyword_id]["ids"].append('user_'+ str(user.id))
                relations = realtion_in_profile(user, keyword_id, relations)
                keywords.append(keyword_id)
            for tweet in tweets:
                if keyword in tweet.full_text:
                    keywords_dict[keyword_id]["ids"].append('tweet_'+ str(tweet.id))
                    relations = realtion_keyword_in_tweet(tweet, keyword_id, relations)
                    keywords.append(keyword_id)

        # go through the users tweets, description, name to find keywords
        counter = 0
        for keys, keyname in keyname.items():
            # go through the users followers description, tweet, and names to 
            # see if it finds any keywords. If it does it adds them to the follower_list
            followers_list, keywords_dict = keyword_search_in_users(keyname, followers, followers_list, keywords_dict, counter)

            # Goes thorugh the users following description, tweet, and names to 
            # see if it finds any keywords. If it does it adds them to the following_list
            following_list, keywords_dict= keyword_search_in_users(keyname, following, following_list, keywords_dict, counter)
            counter += 1 

        # ensure that if not followers or following where found through the keywords
        # to add however many followers are left. 
        followers_list = add_followers_or_following(followers_list)
        following_list = add_followers_or_following(following_list)

        relations = get_relations_followers_following(relations, followers_list, user)
        relations = get_relations_followers_following(relations,following_list, user)

        # Structure to store the data
        User_Structure = {
            "username": user.screen_name,
            "user_id": 'user_' + str(user.id), 
            "followers": followers_list,
            "followees": following_list, #['user_' + str(followees_id) for followees_id in client.get_friends_ids(user.id,user.screen_name,30)]
            "profile_pic": user.profile_image_url,
            "profile_text": user.description,
            "posts": ['tweet_' + tweet.id for tweet in tweets]  ,
            "keywords": keywords
        }
        
        # go through each tweet in users tweet 
        for tweet in tweets:
            tweet_id_name = 'tweet_' + str(tweet.id)

            # check to see if tweet id not in post_strucutre if not create one for the tweet
            if tweet_id_name not in posts:

                # extract the post structure form the tweet and comments 
                original_pic_id_num = pic_id_num_starter
                post_structure, picture_structure, pic_id_num_starter, relations = extract_tweet_picture_structure(user, tweet, pic_id_num_starter, relations)
                posts[tweet_id_name] = post_structure
                if picture_structure:
                    pictures['pic_' + str(original_pic_id_num)] = picture_structure

            replies_in_post = post_structure['comments']
            replies_in_post = [reply.replace('tweet_','') for reply in replies_in_post]
            if replies_in_post:

                for reply in replies_in_post:
                    reply = client.get_tweet_by_id(reply)
                    reply_id_name = 'tweet_' +str(reply.id)

                    if reply_id_name not in posts:
                        original_pic_id_num = pic_id_num_starter
                        post_structure, picture_structure, pic_id_num_starter, relations = extract_tweet_picture_structure(reply.user, reply, pic_id_num_starter, relations)
                        posts[reply_id_name] = post_structure
                        
                        if picture_structure:
                            pictures['pic_' + str(original_pic_id_num)] = picture_structure

            
        # save the user to the dictionary
        user_id_name = 'user_'+ user.id
        if user_id_name not in users:
            users[user_id_name] = User_Structure


        # list that contains all data structures 
        data_structure_for_twitter = [users, posts, comments, pictures, keywords, relations]
        save_json(users, save_path_list[0])
        save_json(posts, save_path_list[1])
        save_json(pictures, save_path_list[3])
        save_json(keywords_dict, save_path_list[4])
        save_json(relations, save_path_list[5])
        print(account_num)
        account_num += 1 
    except TooManyRequests as e:
        print('Rate limit exceeced')
        break


c:\Users\16822\miniconda3\Lib\site-packages\twikit\client.py:3084: UserWarning: Some followers are excluded because "Quality Filter" is enabled. To get all followers, turn off it in the Twitter settings.
  warnings.warn(


1
2


IndexError: list index out of range

In [ ]:
print(users)
# TODO: Test to see how many unsers to pull, guess 10 time.sleep( about 7.5 - 15 min) in order to not exceed time rates.  